In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import library

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#from google.colab.patches import cv2_imshow

# Part1: goodFeaturesToTrack
- Fill the missing part (denoted as ```fill here```) of the code
- We provide procedure comments for complete the function

In [2]:
def goodFeaturesToTrack(image, maxCorners=100, qualityLevel=0.03, blocksize=7):

    # Image bluring wih averaging filter
    # Only cv2.filter2D is allowed for convolution operation!
    #노이즈를 줄이고 image를 부드럽게 만드는 과정 이를 통해 corner detection을 더 잘 되게 만듬
    image = cv2.filter2D(image, -1, np.ones((blocksize, blocksize), np.float32) / (blocksize * blocksize))

    # Compute gradients
    Ix = cv2.filter2D(image, -1, np.array([[-1, 0, 1],
                                           [-2, 0, 2],
                                           [-1, 0, 1]], dtype=np.float32)) * (1 / 8)
    Iy = cv2.filter2D(image, -1, np.array([[-1, -2, -1],
                                           [0, 0, 0],
                                           [1, 2, 1]], dtype=np.float32)) * (1 / 8)

    # Compute products of gradients at each pixel
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Compute the sums of products of gradients in local windows
    Sxx = cv2.filter2D(Ixx, -1, np.ones((blocksize, blocksize)))
    Syy = cv2.filter2D(Iyy, -1, np.ones((blocksize, blocksize)))
    Sxy = cv2.filter2D(Ixy, -1, np.ones((blocksize, blocksize)))

    # Compute the determinant and trace of the matrix M for each pixel
    detM = (Sxx * Syy) - (Sxy * Sxy)
    traceM = Sxx + Syy

    # Compute the Harris response with detM and traceM
    k = 0.04  # Harris detector free parameter
    harris_response = detM - k * (traceM ** 2)

    # Threshold the Harris response to find candidate corners
    corners = np.argwhere(harris_response > (qualityLevel * harris_response.max()))

    # Sort the corners by Harris response in descending order
    sorted_corners = corners[np.argsort(harris_response[corners[:, 0], corners[:, 1]])[::-1]]

    # Keep the top 'maxCorners' corners
    selected_corners = sorted_corners[:maxCorners]

    final_corners = np.array(selected_corners)
    final_corners = final_corners.reshape(-1, 1, 2)

    return final_corners

# Part2: Optical flow with Lukas-Kanade
- Fill the missing part (denoted as ```fill here```) of the code
- We provide procedure comments for complete the function

In [3]:

def optical_flow(old_frame, new_frame, window_size, min_quality):

    feature_list = goodFeaturesToTrack(old_frame, max_corners, min_quality, blocksize)

    w = int(window_size/2)

    # Normalize
    old_frame = old_frame / 255
    new_frame = new_frame / 255

    # Convolve to get gradients w.r.to X, Y and T dimensions
    kernel_x = np.array([[-1, 0, 1],
                         [-2, 0, 2],
                         [-1, 0, 1],], dtype=np.float32) * (1/8)
    kernel_y = np.array([[-1, -2,-1],
                         [0, 0, 0],
                         [1, 2, 1]], dtype=np.float32) * (1/8)
    #아래는 time gradients
    kernel_t = np.array([[1, 1, 1],
                         [1, 1, 1],
                         [1, 1, 1]], dtype=np.float32) * (1/9)

    # cv2.filter2D is allowed for convolution!
    fx =  (cv2.filter2D(old_frame, -1, kernel_x) + cv2.filter2D(new_frame, -1, kernel_x)) 
    fy =  (cv2.filter2D(old_frame, -1, kernel_y) + cv2.filter2D(new_frame, -1, kernel_y))
    #2개의 프레임차이를 계산하여 변화를 계산하는 것
    ft =  (cv2.filter2D(new_frame, -1, kernel_t) + cv2.filter2D(old_frame, -1, kernel_t))

    u = np.zeros(old_frame.shape)
    v = np.zeros(old_frame.shape)

    for feature in feature_list:  # for every corner
        i, j = feature.ravel()  # get cordinates of the corners (i,j).
        i, j = int(i), int(j)  # i,j are floats initially so convert to integer type

        #윈도우 프레임 중심좌표에서 차이만큼 보는거 
        I_x = fx[i - w: i + w + 1, j - w: j + w + 1].flatten()
        I_y = fy[i - w: i + w + 1, j - w: j + w + 1].flatten()
        I_t = ft[i - w: i + w + 1, j - w: j + w + 1].flatten()

        b = np.reshape(I_t, (I_t.shape[0], 1))
        A = np.vstack((I_x, I_y)).T

        U = np.matmul(np.linalg.pinv(A), b)  # Solving for (u,v) i.e., U => A * U = b 푸는거 따라서 U는 A 역행렬에 b를 곱해서 구할수 있음

        u[i, j] = U[0][0]
        v[i, j] = U[1][0]

    return (u, v)


# Main function
- If part1 and part2 were filled properly, the 'output.avi' will be generated!
- For google colab, as cv2.imshow() is not provided, so please use cv2_imshow (google.colab.patches) instead  

In [ ]:
cap = cv2.VideoCapture('/Users/parkjunseo/Desktop/Hanyang/4학년 2학기/컴퓨터비전개론/assignment/ICPBL/slow.mp4')

# Take first frame and find corners in it
ret, old_frame = cap.read()

# Width and height of the file to save
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# 'output.mp4' will be generated!
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4',  fourcc, 30.0, (int(width), int(height)))

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Shi Tomasi parameter
max_corners = 100
min_quality = 0.3
blocksize = 7
p0 = goodFeaturesToTrack(old_frame, max_corners, min_quality, blocksize)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret, current_frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    U, V = optical_flow(old_gray, frame_gray, 15, 0.03)
    for i in range(current_frame.shape[0]):
        for j in range(current_frame.shape[1]):
            u, v = U[i][j], V[i][j]
            if u and v:
                mask = cv2.line(mask, (j, i), (int(round(j + u)), int(round(i + v))), (0, 255, 0), 2)
                frame = cv2.arrowedLine(current_frame, (j, i), (int(round(j + u)), int(round(i + v))), (0, 255, 0), thickness=2)
                current_frame = cv2.add(current_frame, mask)

    # Display the frame with optical flow vectors
    cv2.imshow('optical flow' ,current_frame)
    out.write(current_frame)
    # Break the loop if 'Esc' key is pressed
    if cv2.waitKey(30) == 27:
        break

    # Set the current frame as the previous frame for the next iteration
    old_gray = frame_gray

# Release the video capture object
cap.release()
out.release()

# Close the plot window when done
plt.close()
cv2.destroyAllWindows()


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
2024-12-03 13:35:48.889 python[30346:25280872] +[IMKClient subclass]: chose IMKClient_Legacy
2024-12-03 13:35:48.889 python[30346:25280872] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


: 